<a href="https://colab.research.google.com/github/Hoon-Hoon-Tiger/PyTorch-Implementations/blob/main/Inception.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Refernce
- https://velog.io/@euisuk-chung/%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98-%ED%8C%8C%EC%9D%B4%ED%86%A0%EC%B9%98%EB%A1%9C-CNN-%EB%AA%A8%EB%8D%B8%EC%9D%84-%EA%B5%AC%ED%98%84%ED%95%B4%EB%B3%B4%EC%9E%90-GoogleNet%ED%8E%B8
- https://deep-learning-study.tistory.com/537

In [ ]:
import torch 
import torch.nn as nn 
import torch.nn.functional as F
from torchsummary import summary

# Define Convolution Blocks

## 1 x 1 Convolution

In [ ]:
def Conv_1(in_channels, out_channels):
    conv1 = nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 1, 1),
        nn.ReLU(inplace=True)
    )
    return conv1

## 1 x 1 Convolution -> 3 x 3 Convolution

In [ ]:
def Conv1_3(in_channels, mid_channels, out_channels):
    conv1_3 = nn.Sequential(
        nn.Conv2d(in_channels, mid_channels, 1, 1),
        nn.ReLU(),
        nn.Conv2d(mid_channels, out_channels, 3, 1, 1),
        nn.ReLU(inplace=True)
    )
    return conv1_3

## 1 x 1 Convolution -> 5 x 5 Convolution

In [ ]:
def Conv1_5(in_channels, mid_channels, out_channels):
    conv1_5 = nn.Sequential(
        nn.Conv2d(in_channels, mid_channels, 1, 1),
        nn.ReLU(),
        nn.Conv2d(mid_channels, out_channels, 5, 1, 2),
        nn.ReLU(inplace=True)
    )
    return conv1_5

## 3 x 3 Maxpooling -> 1 x 1 Convolution

In [ ]:
def Max3_Conv1(in_channels, out_channels):
    max3_conv1 = nn.Sequential(
        nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
        nn.Conv2d(in_channels, out_channels, 1, 1),
        nn.ReLU(inplace=True)
    )
    return max3_conv1

# Define inception Module
- 앞에서 정의한 Conv Block들을 이용하여 concat해주는 단계

In [ ]:
class inception_module(nn.Module):
    def __init__(self, in_channels, out_channels_1, mid_channels_3, out_channels_3, mid_channels_5, out_channels_5, pool_channels):
        super(inception_module, self).__init__()
        
        # 1*1 conv
        self.conv_1 = Conv_1(in_channels, out_channels_1)
        
        # 1*1 conv -> 3*3 conv
        self.conv1_3 = Conv1_3(in_channels, mid_channels_3, out_channels_3)
        
        # 1*1 conv -> 5*5 conv
        self.conv1_5 = Conv1_5(in_channels, mid_channels_5, out_channels_5)
        
        # 3*3 Max -> 1*1 conv
        self.max3_1= Max3_Conv1(in_channels, pool_channels)
        
    def forward(self, x):
        out_1 = self.conv_1(x)
        out_2 = self.conv1_3(x)
        out_3 = self.conv1_5(x)
        out_4 = self.max3_1(x)
        
        #concatenation
        output = torch.cat([out_1, out_2, out_3, out_4], 1)
        return output

In [ ]:
class GoogLeNet(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(GoogLeNet, self).__init__()
        
        self.training = True
        
        self.layer_1 = nn.Sequential(
            nn.Conv2d(in_channels, 64, 7, 2, 3),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.LocalResponseNorm(2),
            nn.Conv2d(64, 64, 1),
            nn.Conv2d(64, 192, 3, 1, 1),
            nn.LocalResponseNorm(2),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        )
        
        self.inception_3a = inception_module(192, 64, 96, 128, 16, 32, 32)
        self.inception_3b = inception_module(256, 128, 128, 192, 32, 96, 64)
        self.maxpool_3 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)   
            
    
        self.inception_4a = inception_module(480, 192, 96, 208, 16, 48, 64)
        self.aux1 = AuxModule(512, num_classes)
        
        self.inception_4b = inception_module(512, 160, 112, 224, 24, 64, 64)
        self.inception_4c = inception_module(512, 128, 128, 256, 24, 64, 64)
        self.inception_4d = inception_module(512, 112, 144, 288, 32, 64, 64)
        self.aux2 = AuxModule(528, num_classes)
        
        self.inception_4e = inception_module(528, 256, 160, 320, 32, 128, 128)
        self.maxpool_4 =nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.inception_5a = inception_module(832, 256, 160, 320, 32, 128, 128)
        self.inception_5b = inception_module(832, 384, 192, 384, 48, 128, 128)
        
        self.avgpool = nn.AvgPool2d(7, 1)
        
        self.dropout = nn.Dropout2d(0.4)
        self.fc = nn.Linear(1024, num_classes)
        
    def forward(self, x):
        
        out = self.layer_1(x)
        
        out = self.inception_3a(out)
        out = self.inception_3b(out)
        out = self.maxpool_3(out)
        
        out = self.inception_4a(out)
        if self.training:
            aux_out1 = self.aux1(out)
            
        out = self.inception_4b(out)
        out = self.inception_4c(out)
        out = self.inception_4d(out)
        if self.training:
            aux_out2 = self.aux2(out)
        
        out = self.inception_4e(out)
        out = self.maxpool_4(out)
        
        out = self.inception_5a(out)
        out = self.inception_5b(out)
        
        out = self.avgpool(out)
        
        out = torch.flatten(out, 1)
        out = self.dropout(out)
        out = self.fc(out)
        if self.training:
            return [out, aux_out1, aux_out2]
        else:
            return out
        
    def set_train(self):
        self.training = True

    def set_eval(self):
        self.training = False
        
class AuxModule(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(AuxModule, self).__init__()
        
        self.avgpool = nn.AdaptiveAvgPool2d((4,4))
        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=1),
            nn.ReLU(inplace=True)
        )
        self.fc = nn.Sequential(
            nn.Linear(4*4*128, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout2d(p=0.7),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.avgpool(x)
        x = self.conv1(x)
        x = torch.flatten(x, 1)
        return self.fc(x) 

In [ ]:
#네트워크 생성
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
batch_size = 3
model = GoogLeNet(3, 10).to(device)
x = torch.randn(3, 3, 224, 224).to(device)
output = model(x)
print(output[0].shape)

cuda:0
torch.Size([3, 10])


/usr/local/lib/python3.8/dist-packages/torch/nn/functional.py:1331: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


In [ ]:
summary(model, (3, 224, 224), device=device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,472
              ReLU-2         [-1, 64, 112, 112]               0
         MaxPool2d-3           [-1, 64, 56, 56]               0
 LocalResponseNorm-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]           4,160
            Conv2d-6          [-1, 192, 56, 56]         110,784
 LocalResponseNorm-7          [-1, 192, 56, 56]               0
         MaxPool2d-8          [-1, 192, 28, 28]               0
            Conv2d-9           [-1, 64, 28, 28]          12,352
             ReLU-10           [-1, 64, 28, 28]               0
           Conv2d-11           [-1, 96, 28, 28]          18,528
             ReLU-12           [-1, 96, 28, 28]               0
           Conv2d-13          [-1, 128, 28, 28]         110,720
             ReLU-14          [-1, 128,

In [ ]:
model

GoogLeNet(
  (layer_1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (3): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (4): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
    (5): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): LocalResponseNorm(2, alpha=0.0001, beta=0.75, k=1.0)
    (7): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (inception_3a): inception_module(
    (conv_1): Sequential(
      (0): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU(inplace=True)
    )
    (conv1_3): Sequential(
      (0): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): Conv2d(96, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
    )
    (conv1_5): Sequential(
      (0): Conv2d(192, 